In [ ]:
#Imports

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.convolutional import Conv2D

In [ ]:
#Global Variables
train_dir = 'data/train/'
new_size = (256,256)

In [ ]:
im_mask = Image.open(train_dir+'3594_mask.png')
im_sat = Image.open(train_dir+'3594_sat.jpg')

resized_mask = im_mask.resize(new_size)
resized_sat = im_sat.resize(new_size)


In [ ]:
resized_sat

In [ ]:
resized_mask

In [ ]:
resized_mask_matrix = np.asarray(resized_mask.convert("L"))
resized_sat_matrix = np.asarray(resized_sat.convert("RGB"))

In [ ]:
from preprocessing import get_patch

In [ ]:
mask_patch = Image.fromarray(resized_mask_matrix.astype('uint8'), 'L')
image_patch = Image.fromarray(resized_sat_matrix.astype('uint8'), 'RGB')

In [ ]:
image_patch

In [ ]:
mask_patch

In [ ]:
#Prepare data
np.random.seed(44)

X_train, Y_train = [], []

for i in range(100):
    mask_patch, im_patch = get_patch(resized_mask_matrix, resized_sat_matrix)
    X_train.append(im_patch)
    Y_train.append(mask_patch)
X_train = np.array(X_train)
Y_train = np.array(Y_train)
print(X_train.shape)

In [ ]:
# Model definition
activation = "relu"
num_layers = 10
loss_function = "logcosh"
optimizer_algorithm = "adam"

In [ ]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop

from losses import bce_dice_loss, dice_loss, weighted_bce_dice_loss, weighted_dice_loss, dice_coeff

In [ ]:
def get_unet_256(input_shape=(256, 256, 3),
                 num_classes=1):
    inputs = Input(shape=input_shape)
    # 256

    down0 = Conv2D(32, (3, 3), padding='same')(inputs)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    # 128

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 32

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)
    # 16

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)
    # 8

    center = Conv2D(1024, (3, 3), padding='same')(down4_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    # center

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    # 16

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    # 128

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    # 256

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0)

    model = Model(inputs=inputs, outputs=classify)

    model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

    return model



In [ ]:
Y_train.reshape(100,32,32,1).shape

In [ ]:
model = get_unet_256(input_shape=(32,32,3),num_classes=1)
model.summary()
model.fit(X_train, Y_train.reshape(100,32,32,1), epochs=100, shuffle=True, verbose=1, batch_size=64)

In [ ]:
# Model

model = Sequential()
model.add(Dense(256, input_shape=(32*32*3,)))
model.add(Activation(activation))
for i in range(num_layers):
    model.add(Dense(256))
    model.add(Activation(activation))
model.add(Dense(32*32))
model.add(Activation(activation))

model.summary()

model.compile(loss=loss_function, optimizer=optimizer_algorithm, metrics=["accuracy"])

In [ ]:
print(Y_train.reshape(100,32*32).shape)
model.fit(X_train.reshape(100,32*32*3), Y_train.reshape(100,32*32), epochs=1000, shuffle=True, verbose=1, batch_size=64)


In [ ]:
Y = model.predict(X_train, batch_size=10000)
k = 0
Y = Y.reshape(100,32,32)
Y = Y > 0.5

In [ ]:
print(Y[2].sum())

In [ ]:
%matplotlib inline
num=10
plt.imshow(Y[num])

In [ ]:
plt.imshow(Y_train[num])

In [ ]:
plt.imshow(X_train[num])

In [ ]:
#Prepare data
np.random.seed(222)

X_train, Y_train = [], []

for i in range(100):
    mask_patch, im_patch = get_patch(resized_mask_matrix, resized_sat_matrix)
    X_train.append(im_patch)
    Y_train.append(mask_patch)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

Y = model.predict(X_train, batch_size=10000)
k = 0
Y = Y.reshape(100,32,32)
Y = Y > 0.5

In [ ]:
%matplotlib inline
num=25
plt.imshow(Y[num])

In [ ]:
plt.imshow(Y_train[num])

In [ ]:
plt.imshow(X_train[num])